In [ ]:
import pandas as pd
import re
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
!pip install nlpaug
import nlpaug.augmenter.word as naw

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv('comments_public.tsv',sep='\t')

In [ ]:
train_data = data[:900]
train_data.head()

,should_ban,comment_text
0,0.0,The picture on the article is not of the actor...
1,1.0,"Its madness. Shes of Chinese heritage, but JAP..."
2,1.0,Fuck You. Why don't you suck a turd out of my ...
3,1.0,God is dead\nI don't mean to startle anyone bu...
4,1.0,THIS USER IS A PLANT FROM BRUCE PERENS AND GRO...


In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
aug = naw.SynonymAug(aug_src='wordnet')

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [ ]:
def augment_text(text):
    augmented = aug.augment(text)
    if isinstance(augmented, list):
        return ' '.join(augmented)
    return augmented

In [ ]:
processed_texts = [preprocess_text(text) for text in train_data['comment_text']]
processed_texts[:3]

['picture article actor tr knight subject article even basic google search turn picture tr knight article written photo continually added article photo article subject photo relevant subject article removed edit others made edit reverted please provide legitimate reason image inclusion article fairuse image tr knight actor subject article found mean added random image used placeholder',
 'madness shes chinese heritage japanese thats impossible like one irish heritage born america would make american would imply person nationality biased born little came madness',
 'fuck dont suck turd asshole think handle dumbfuck eh']

In [ ]:
augmented_texts = [augment_text(text) for text in processed_texts]

In [ ]:
combined_texts = processed_texts + augmented_texts

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(combined_texts)
print(X[:3].toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
y = train_data['should_ban'].tolist() * 2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ

In [ ]:
from sklearn.linear_model import LogisticRegression

param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs', 'sag', 'saga'],
    'penalty': ['l2', 'l1', 'none'],
    'max_iter': [100, 200, 300]
}

lr_model = LogisticRegression(max_iter=300)
grid_search_lr = GridSearchCV(estimator=lr_model, param_grid=param_grid_lr, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)
best_lr_model = grid_search_lr.best_estimator_

y_pred_lr = best_lr_model.predict(X_test)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_lr)}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
324 fits failed out of a total of 648.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1194, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/

Logistic Regression Accuracy: 0.9277777777777778


ГРАДИЕНТНЫЙ БУСТИНГ

In [ ]:
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}
gb_model = GradientBoostingClassifier()
grid_search_gb = GridSearchCV(estimator=gb_model, param_grid=param_grid_gb, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(X_train, y_train)
best_gb_model = grid_search_gb.best_estimator_

y_pred_gb = best_gb_model.predict(X_test)
print(f"Gradient Boosting Accuracy: {accuracy_score(y_test, y_pred_gb)}")
print(f"Best parameters for GB: {grid_search_gb.best_params_}")

Gradient Boosting Accuracy: 0.8759259259259259
Best parameters for GB: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


+ RANDOM FOREST И SVC

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Support Vector Machine Classifier
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm)}")

Random Forest Accuracy: 0.8777777777777778
SVM Accuracy: 0.9222222222222223


СВЁРТОЧНАЯ НС

In [ ]:
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(processed_texts)
X_seq = tokenizer.texts_to_sequences(processed_texts)
X_padded = pad_sequences(X_seq, maxlen=max_len)

X_train_padded, X_test_padded, y_train_cnn, y_test_cnn = train_test_split(X_padded, train_data['should_ban'], test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train_cnn, epochs=50, batch_size=32, validation_data=(X_test_padded, y_test_cnn))

loss, accuracy = model.evaluate(X_test_padded, y_test_cnn)
print(f"Test accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.5108 - loss: 0.6953 - val_accuracy: 0.5833 - val_loss: 0.6849
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.5863 - loss: 0.6831 - val_accuracy: 0.5778 - val_loss: 0.6813
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.6078 - loss: 0.6584 - val_accuracy: 0.5833 - val_loss: 0.6689
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.7670 - loss: 0.5923 - val_accuracy: 0.6111 - val_loss: 0.6240
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.8580 - loss: 0.4691 - val_accuracy: 0.6111 - val_loss: 0.6792
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9493 - loss: 0.3658 - val_accuracy: 0.6667 - val_loss: 0.6205
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9899 - loss: 0.3399 - val_accuracy: 0.6389 - val_loss: 0.6650
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9950 - loss: 0.2960 - val_accuracy: 0.6667 - v

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.5))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train_cnn, epochs=50, batch_size=32, validation_data=(X_test_padded, y_test_cnn))

loss, accuracy = model.evaluate(X_test_padded, y_test_cnn)
print(f"Test accuracy: {accuracy}")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.4991 - loss: 0.6940 - val_accuracy: 0.5833 - val_loss: 0.6860
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.5787 - loss: 0.6818 - val_accuracy: 0.5778 - val_loss: 0.6872
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.6004 - loss: 0.6615 - val_accuracy: 0.5611 - val_loss: 0.6728
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6609 - loss: 0.6226 - val_accuracy: 0.6111 - val_loss: 0.6502
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.7853 - loss: 0.5248 - val_accuracy: 0.6778 - val_loss: 0.5905
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.8685 - loss: 0.3257 - val_accuracy: 0.6944 - val_loss: 0.6483
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.9175 - loss: 0.2044 - val_accuracy: 0.6944 - val_loss: 0.7778
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.9499 - loss: 0.1688 - val_accuracy: 0.6833 - val_loss: 0.

In [ ]:
# Assuming you have the remaining 100 elements
remaining_data = train_data[900:]  # Adjust the slice as necessary
remaining_texts = remaining_data['comment_text'].tolist()

# Preprocess the remaining texts
processed_remaining_texts = [preprocess_text(text) for text in remaining_texts]

# Tokenize and pad the sequences
remaining_seq = tokenizer.texts_to_sequences(processed_remaining_texts)
remaining_padded = pad_sequences(remaining_seq, maxlen=max_len)

# Make predictions on the remaining data
predicted_probabilities = best_lr_model.predict(remaining_padded)

# Convert probabilities to classes using a threshold (0.5 for binary classification)
predicted_classes = (predicted_probabilities > 0.5).astype(int)

# Output the results
for text, prob, pred_class in zip(remaining_texts, predicted_probabilities, predicted_classes):
    print(f"Text: {text}")
    print(f"Predicted Probability: {prob[0]:.4f}, Predicted Class: {pred_class[0]}")
    print("---")


ValueError: Found array with 0 sample(s) (shape=(0, 100)) while a minimum of 1 is required by LogisticRegression.

In [ ]:
import numpy as np

remaining_data = data[900:]
remaining_texts = remaining_data['comment_text'].tolist()
remaining_tfidf = tfidf_vectorizer.transform(remaining_texts)

predicted_probabilities = best_lr_model.predict_proba(remaining_tfidf)[:, 1]

results_df = pd.DataFrame({
    'comment_text': remaining_texts,
    'predicted_probability': predicted_probabilities
})

np.save('probs.npy', predicted_probabilities)
predicted_probabilities

array([0.49803211, 0.65840672, 0.99222159, 0.23034455, 0.92008328,
       0.95518169, 0.68459271, 0.29105247, 0.41817126, 0.22168647,
       0.35755657, 0.54157278, 0.34357644, 0.32386665, 0.46484506,
       0.39099791, 0.55448181, 0.41716299, 0.69986462, 0.47268843,
       0.46301015, 0.85741993, 0.66246169, 0.17835816, 0.45980016,
       0.44769975, 0.79868102, 0.67724745, 0.58530691, 0.65843154,
       0.30049062, 0.70800614, 0.38524904, 0.28467411, 0.13766357,
       0.34971189, 0.59015626, 0.42126908, 0.82926292, 0.12559599,
       0.94192043, 0.98054382, 0.35035126, 0.51677585, 0.42803888,
       0.46686696, 0.18801058, 0.42488858, 0.80013521, 0.25772086,
       0.65319044, 0.34913727, 0.58761854, 0.31853811, 0.30848312,
       0.49498668, 0.54743775, 0.82350813, 0.58275126, 0.65158124,
       0.57945779, 0.8773241 , 0.45268573, 0.12984913, 0.86611819,
       0.98734489, 0.75870762, 0.75990305, 0.49100584, 0.68718955,
       0.99594349, 0.59408883, 0.31642097, 0.52479227, 0.38184